In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import cv2
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import optim
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pickle
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

import segmentation_models_pytorch as smp

torch.backends.cudnn.benchmark = True
scaler = torch.cuda.amp.GradScaler()

/Users/silas/work/nest/nest_env/lib/python3.12/site-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(f"Using {device}")

Using mps


In [3]:
run_test_cases = False # Set to True if debugging is required
cloud_labels = ["Flower", "Gravel", "Fish", "Sugar"] # All possible labels for the clouds

# Original Image resolutions
in_res_y = 1400
in_res_x = 2100

# New Image resolutions
new_res_y = 512
new_res_x = 768

# data directories
test_dir = "./test_images"
train_dir = "./train_images"

# Training params
num_filters = 32 # Number of filters in first conv layer  
num_train_images = 5040
num_test_images = 256

batch_size = 8
num_epochs = 60
lr = 3e-4
weight_decay = 1e-4

In [4]:
# Create Dataframe
df = pd.read_csv('train.csv')
df[['Image', 'Label']] = df['Image_Label'].str.split('_', expand=True)

# Find images with at least one mask
df_nonempty = df.groupby("Image")["EncodedPixels"].apply(lambda x: x.notna().any()).reset_index()
df_nonempty = df_nonempty[df_nonempty["EncodedPixels"] == True]
valid_images = df_nonempty["Image"].unique().tolist()

image_names = [img for img in sorted(os.listdir(train_dir)) if img in valid_images]

# remove broken images
def find_broken_images(img_dir):

    # Load possibly saved broken file
    if os.path.exists(f"{img_dir}_broken.pkl"):
        with open(f"{img_dir}_broken.pkl", "rb") as f:
            broken = pickle.load(f)
        return broken
     
    broken = []
    for f in os.listdir(img_dir):
        path = os.path.join(img_dir, f)
        img = cv2.imread(path)
        if img is None:
            broken.append(f)

    with open(f"{img_dir}_broken.pkl", "wb") as f:
        pickle.dump(broken, f)

    return broken

#broken_train = find_broken_images("./train_images")
#broken_test = find_broken_images("./test_images")

#print("Broken train images:", broken_train)
#print("Broken test images:", broken_test)

# Remove them from your image list
#image_names = [f for f in image_names if f not in broken_train]

train_images = image_names[:num_train_images]
test_images = image_names[num_train_images:num_train_images+num_test_images]

if run_test_cases:
    print(df[['Image', 'Label', 'EncodedPixels']].head(8))
    print()
    print(df['Image'].unique()[:10])

In [5]:
# Get labels and rle from image name
def get_labels_rle(image_name: str, df) -> list:
    rles = df[df['Image'] == image_name]['EncodedPixels'].to_list()
    labels = df[df['Image'] == image_name]['Label'].to_list()
    return rles, labels

# Debugging
if run_test_cases:

    # Get Files
    test_train_images = os.listdir(train_dir)[:4]
    print(f"Train images: {test_train_images}")

    for image in test_train_images:
        rles, labels = get_labels_rle(f"{image}", df)
        for rle, label in zip(rles, labels):
            print(f"Label: {label} \n rle: {rle} \n")

In [6]:
# Convert rle mask encoding into 2D arrays
def rle_to_array(rle_list: list) -> np.array:

    # Create empty array for
    array = np.zeros(in_res_y * in_res_x)

    # Skip if cloud formation is not on picture
    if not rle_list or pd.isna(rle_list):
        mask = array.reshape((in_res_x, in_res_y), order="A").T
        return mask
    
    rle_array = np.array(list(map(int, rle_list.split())), dtype=int)
    start_pixels = rle_array[::2] - 1 # Offset because pixel 1 is arr position 0
    num_pixels = rle_array[1::2]

    # Create 2D mask
    for start_pixel, num_pixels in zip(start_pixels, num_pixels): # Format is [start_idx_0, num_pixels_0 ...]
        array[start_pixel:start_pixel+num_pixels] = 1.0
    
    # Reshape
    mask = array.reshape((in_res_x, in_res_y), order="A").T # 2D array of [Height, Width]

    return mask

# For debugging
if run_test_cases:

    # Get Files
    test_train_images = os.listdir(train_dir)[:2]
    print(f"Train images: {test_train_images}")

    # Plot files
    for image_name in test_train_images:
        img = cv2.imread(f"{train_dir}/{image_name}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (new_res_x, new_res_y), interpolation=cv2.INTER_AREA)

        rles, labels = get_labels_rle(image_name, df)

        # Raw Image
        plt.imshow(img)
        plt.show()

        for rle, label in zip(rles, labels):

            # Masked Image
            mask = rle_to_array(rle)
            mask = cv2.resize(mask, (new_res_x, new_res_y), interpolation=cv2.INTER_NEAREST)
            print(np.unique(mask))
            print(f"titel: {label}")
            plt.imshow(mask, cmap="grey", vmin=0.0, vmax=1.0)
            #plt.imshow(mask[:, :, None].repeat(3, axis=-1)*img, cmap="grey", vmin=0.0, vmax=1.0)
            plt.show()
        print("---------------------------------")

In [7]:
def dice_coef(preds, target, eps=1e-6):
    # [B, 4, H, W]

    preds = torch.sigmoid(preds)
    overlap = (preds * target).sum((2,3))

    dice = (2. * overlap + eps) / (preds.sum((2,3)) + target.sum((2,3)) + eps)

    return dice.mean()

In [8]:
def dice_loss(preds, target, eps=1e-6):
    return 1 - dice_coef(preds, target, eps=eps)

In [9]:
bce_loss = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([5.0]).to(device))

In [10]:
def loss_fn(preds, target, split=0.5, eps=1e-6):
    return split * dice_loss(preds, target, eps) + (1 - split) * bce_loss(preds, target)

In [ ]:
train_transform = A.Compose([
    # --- geometric transforms ---
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.Transpose(p=0.2),

    # --- photometric transforms ---
    A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.3),
    A.HueSaturationValue(hue_shift_limit=3, sat_shift_limit=5, val_shift_limit=5, p=0.3),
    
    A.ElasticTransform(alpha=50, sigma=8, alpha_affine=8, p=0.3),
    A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
    A.GaussNoise(var_limit=(5, 20), p=0.3),

    # --- spatial transforms ---
    A.ShiftScaleRotate(
        shift_limit=0.05,
        scale_limit=0.1,
        rotate_limit=15,
        border_mode=cv2.BORDER_REFLECT,
        p=0.5
    ),

    # --- normalization ---
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),

    # --- ensure correct shape ---
    A.Resize(height=new_res_y, width=new_res_x),

    # --- to tensor ---
    ToTensorV2(),
])

val_transform = A.Compose([
    # --- normalization ---
    A.Normalize(mean=(0.485, 0.456, 0.406), 
                std=(0.229, 0.224, 0.225)),

    # --- ensure correct shape ---
    A.Resize(height=new_res_y, width=new_res_x),

    # --- to tensor ---
    ToTensorV2(),
])



/var/folders/31/lhm9rkfd5mq5nqs778m806v40000gn/T/ipykernel_70940/112616903.py:12: UserWarning: Argument(s) 'alpha_affine' are not valid for transform ElasticTransform
  A.ElasticTransform(alpha=50, sigma=8, alpha_affine=8, p=0.3),
/Users/silas/work/nest/nest_env/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


In [ ]:
class ImageDataset(Dataset):
    def __init__(self, data_frame, img_dir, image_names, transform=None):
        self.img_dir = img_dir
        self.image_names = image_names
        self.data_frame = data_frame
        self.transform = transform

    def __len__(self):
        return len(self.image_names)
    
    def __getitem__(self, idx): 
        image_name = self.image_names[idx]

        # --- Load image ---
        image = cv2.imread(f"{self.img_dir}/{image_name}")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # --- Downsample image ---
        image = cv2.resize(image, (new_res_x, new_res_y), interpolation=cv2.INTER_AREA)
        image = image.astype(np.float32)

        rles, _ = get_labels_rle(image_name, self.data_frame)

        # --- Create mask ---
        mask = np.zeros((4, in_res_y, in_res_x), dtype=np.float32)
        for i, rle in enumerate(rles):
            single_mask = rle_to_array(rle)
            mask[i, :, :] = single_mask

        # --- Downsample mask per channel ---
        mask_resized = np.zeros((4, new_res_y, new_res_x), dtype=np.float32)
        for i in range(4):
            mask_resized[i, :, :] = cv2.resize(
                    mask[i],
                    (new_res_x, new_res_y),
                    interpolation=cv2.INTER_NEAREST
                )
        mask = mask_resized

        image = image.astype(np.uint8)

        # --- Transformations ---
        if self.transform:
            transformed = self.transform(
                image=image,
                mask=mask.transpose(1, 2, 0)
            )
            image = transformed["image"]
            mask = transformed["mask"].permute(2, 0, 1)  # [C, H, W]
        else:
            # manually convert only if no transform applied
            image = torch.from_numpy(image).permute(2, 0, 1).float()
            mask = torch.from_numpy(mask).float()

        return image, mask


In [13]:
class ConvolutionBlock(nn.Module):
    def __init__(self, in_channels, out_channels, conv_kernel_size, padding):
        super().__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, conv_kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, conv_kernel_size, padding=padding),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
            )

    def forward(self, x):
        output = self.conv_block(x)
        return output # [batch, out_channels, H_out, W_out]


In [14]:
class PoolBlock(nn.Module):
    def __init__(self, downsample):
        super().__init__()

        self.pool = nn.MaxPool2d(downsample)

    def forward(self, x):
        output = self.pool(x)
        return output

In [15]:
class UpSampleBlock(nn.Module):
    def __init__(self, channels, upsample):
        super().__init__()

        self.up_sample_block = nn.Sequential(
            nn.Upsample(scale_factor=upsample, mode='bilinear', align_corners=False),
            nn.Conv2d(channels, channels // upsample, kernel_size=1)
        )

    def forward(self, x):
        upsample_block = self.up_sample_block(x)
        return upsample_block

In [16]:
class Network(nn.Module):
    def __init__(self, filters):
        super().__init__()

        # in_channels, out_channels, conv_kernel_size, padding, pool_kernel_size
        self.encoder_layer_0 = ConvolutionBlock(3, filters, 3, 1)
        self.encoder_layer_1 = ConvolutionBlock(filters, 2*filters, 3, 1)
        self.encoder_layer_2 = ConvolutionBlock(2*filters, 4*filters, 3, 1)

        self.pool_block_0 = PoolBlock(downsample=2)
        self.pool_block_1 = PoolBlock(downsample=2)
        self.pool_block_2 = PoolBlock(downsample=2)

        self.bottle_neck = nn.Conv2d(4*filters, 8*filters, 3, padding=1)

        # in_channels, out_channels, conv_kernel_size, stride
        self.decoder_layer_2 = ConvolutionBlock(8*filters, 4*filters, 3, 1)
        self.decoder_layer_1 = ConvolutionBlock(4*filters, 2*filters, 3, 1)
        self.decoder_layer_0 = ConvolutionBlock(2*filters, filters, 3, 1)

        self.up_sample_block2 = UpSampleBlock(8 * filters, upsample=2)
        self.up_sample_block1 = UpSampleBlock(4 * filters, upsample=2)
        self.up_sample_block0 = UpSampleBlock(2 * filters, upsample=2)

        self.output_layer = nn.Conv2d(filters, 4, 1)

    def forward(self, x): # [Batch, Color, Height, Width]
        
        enc0 = self.encoder_layer_0(x) # [B, num_filters, H, W]
        pool0 = self.pool_block_0(enc0) # [B, num_filters, H / 2, W / 2]

        enc1 = self.encoder_layer_1(pool0) # [B, 2 * num_filters, H / 2, W / 2]
        pool1 = self.pool_block_1(enc1) # [B, 2 * num_filters, H / 4, W / 4]

        enc2 = self.encoder_layer_2(pool1) # [B, 4 * num_filters, H / 4, W / 4]
        pool2 = self.pool_block_2(enc2) # [B, 4 * num_filters, H / 8, W / 8]

        bottle_neck = self.bottle_neck(pool2) # [B, 4 * num_filters, H / 4, W / 4]

        up2 = self.up_sample_block2(bottle_neck) # [B, 4 * num_filters, H / 4, W / 4]
        concat2 = torch.cat([up2, enc2], dim=1) # [B, 8 * num_filters, H / 4, W / 4]
        dec2 = self.decoder_layer_2(concat2) # [B, 4 * num_filters, H / 4, W / 4]

        up1 = self.up_sample_block1(dec2) # [B, 2 * num_filters, H / 2, W / 2]
        concat1 = torch.cat([up1, enc1], dim=1) # [B, 4 * num_filters, H / 2, W / 2]
        dec1 = self.decoder_layer_1(concat1) # [B, 2 * num_filters, H / 2, W / 2]

        up0 = self.up_sample_block0(dec1) # [B, num_filters, H, W]
        concat0 = torch.cat([up0, enc0], dim=1) # [B, 2 * num_filters, H, W]
        dec0 = self.decoder_layer_0(concat0) # [B, num_filters, H, W]

        logits = self.output_layer(dec0) # [B, 4, H, W]

        return logits

In [17]:
# Datasets and DataLoader
train_dataset = ImageDataset(df, train_dir, train_images, transform=train_transform)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=0)

test_dataset = ImageDataset(df, train_dir, test_images, transform=val_transform)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True, num_workers=0)

In [18]:
model = smp.Unet(
    encoder_name="efficientnet-b4",        # backbone
    encoder_weights="imagenet",            # use pretrained ImageNet weights
    in_channels=3,                         # RGB images
    classes=4,                             # 4 cloud types
    decoder_use_batchnorm=True
)

# Inject dropout layers into decoder blocks
for i, block in enumerate(model.decoder.blocks):
    model.decoder.blocks[i].add_module(f"dropout_{i}", nn.Dropout2d(p=0.5))

In [19]:
#model = Network(num_filters).to(device)

In [20]:
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model = torch.nn.DataParallel(model)

model = model.to(device)

In [21]:
optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)

/Users/silas/work/nest/nest_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [22]:
# Albumentations ImageNet stats
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])

def denormalize(img_tensor):
    img = img_tensor.permute(1, 2, 0).cpu().numpy()
    img = std * img + mean  # undo normalization
    img = np.clip(img, 0, 1)
    return img

if run_test_cases:

    for image, mask in train_dataloader:
        # Move batch tensors to device
        image, mask = image.to(device), mask.to(device)

        # Loop over images in the batch
        for img, msk in zip(image, mask):
            # Convert tensor -> NumPy for display
            
            print(np.sum(img.cpu().numpy()))
            if np.sum(img.cpu().numpy()) == 0:
                print("Error")

            print("image")
            plt.figure()
            plt.imshow(denormalize(img))
            plt.show(block=False)   # prevents overwriting
            plt.pause(0.1)          # forces GUI flush

            for m in msk:
                print("mask")
                plt.imshow(m.cpu().numpy(), cmap="grey", vmin=0.0, vmax=1.0)
                plt.show()


In [ ]:
# Training loop
for epoch in range(num_epochs):

    # Training
    model.train()
    train_loss = 0.0

    # Dynamic weigthing of Dice and BCE loss 
    split = 1 - (epoch / num_epochs) * 0.35
    
    for image, mask in train_dataloader:
        image, mask = image.to(device), mask.to(device)

        with torch.cuda.amp.autocast():
            preds = model(image)
            loss = loss_fn(preds, mask, split)

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        train_loss += loss.item()
    train_loss /= len(train_dataloader)

    # Validation
    model.eval()
    val_loss = 0.0
    dice = 0.0

    with torch.no_grad():
        for image, mask in test_dataloader:
            image, mask = image.to(device), mask.to(device)

            preds = model(image)
            loss = loss_fn(preds, mask, split)

            val_loss += loss.item()

            #hard_preds = torch.where(preds > 0.5, 1.0, 0.0)
            dice += dice_coef(preds, mask)
        val_loss /= len(test_dataloader)
        scheduler.step(val_loss)
        dice /= len(test_dataloader)

    # Print metrics
    print(f"Epoch: {epoch}")
    print(f"Train loss: {train_loss:.4f}")
    print(f"Val loss: {val_loss:.4f}")
    print(f"Dice coefficient: {dice:.4f}")
    print()

(512, 768, 3) uint8 253 0
(4, 512, 768) float32

(512, 768, 3) uint8 255 0
(4, 512, 768) float32

(512, 768, 3) uint8 254 0
(4, 512, 768) float32

(512, 768, 3) uint8 252 0
(4, 512, 768) float32

(512, 768, 3) uint8 254 0
(4, 512, 768) float32

(512, 768, 3) uint8 253 0
(4, 512, 768) float32

(512, 768, 3) uint8 250 0
(4, 512, 768) float32

(512, 768, 3) uint8 252 0
(4, 512, 768) float32



/Users/silas/work/nest/nest_env/lib/python3.12/site-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
